## 네이버 지식IN - 추석선물

In [39]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import quote
from tqdm import tqdm

- 지식인 추석 선물 검색결과 화면

In [40]:
url = f'https://kin.naver.com/search/list.naver?query={quote("추석 선물")}&page=1'
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [41]:
lis = soup.select('.basic1 > li')
len(lis)

10

- 답변 갯수가 5개보다 큰 항목을 선택

In [42]:
li = lis[7]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=6&dirId=6130103&docId=454639453&qb=7LaU7ISdIOyEoOusvA==&enc=utf8&section=kin&rank=8&search_sort=0&spq=0'

In [43]:
driver.get(sub_url)
time.sleep(2)

In [44]:
# 더 보기 클릭 횟수 구하기
answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
click_count = (int(answer_num) - 1) // 5
answer_num, click_count

('8', 1)

In [45]:
#  더 보기 횟수 만큼 클릭하기
for _ in range(click_count):
    driver.find_element(By.ID, 'nextPageButton').click()
    time.sleep(2)
    

In [46]:
# 펼쳐진 화면을 BeautifulSoup으로 파싱
sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
divs = sub_soup.select('.answer-content__list._answerList > div')
len(divs)

8

In [47]:
# 답변 목록에서 개별을 선택학 글을 추출하기
div = divs[0]
ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
len(ps)

45

In [48]:
answer = ''
for p in ps:
    text = p.get_text().strip()
    if text == '\u200b' or text == '':
        continue
    answer += text + '\n'
print(answer[:100])

많은 인원수 준비 하실려면 부담이 크시겠어요
연령대도 다양하시고
남녀노소 가리지 않고 해드릴수 있는 선물을 추천 해드릴려고 합니다
곧 풍성한 추석이 몇일 남지 않았어요
부모님추석선


In [49]:
# 답변 목록에 있는 모든 답변을 선택해서 글을 추출하기
answer_list = []
for div in divs:
    ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
    answer = ''
    for p in ps:
        text = p.get_text().strip()
        if text == '\u200b' or text == '':
            continue
        answer += text + '\n'
    answer_list.append(answer)


In [50]:
len(answer_list)

8

In [51]:
driver.back()

- 한 화면에 있는 10개의 글에서 모든 답변을 추출하기

In [65]:
answer_list = []
for li in lis:
    sub_url = li.select_one('a._searchListTitleAnchor')['href']
    driver.get(sub_url)
    time.sleep(2)

    answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
    click_count = (int(answer_num) - 1) // 5
    print(click_count, end=' ')
    for _ in range(click_count):
        driver.find_element(By.ID, 'nextPageButton').click()
        time.sleep(2)
    sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
    divs = sub_soup.select('.answer-content__list._answerList > div')
    for div in divs:
        ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
        answer = ''
        for p in ps:
            text = p.get_text().strip()
            if text == '\u200b' or text == '':
                continue
            answer += text + '\n'
        answer_list.append(answer)
    driver.back()
    time.sleep(2)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=117.0.5938.89)
Stacktrace:
	GetHandleVerifier [0x00007FF6ED1178A2+54818]
	(No symbol) [0x00007FF6ED086AD2]
	(No symbol) [0x00007FF6ECF3DA3B]
	(No symbol) [0x00007FF6ECF1DEE8]
	(No symbol) [0x00007FF6ECFA3237]
	(No symbol) [0x00007FF6ECFB6C6F]
	(No symbol) [0x00007FF6ECF9E883]
	(No symbol) [0x00007FF6ECF73691]
	(No symbol) [0x00007FF6ECF748D4]
	GetHandleVerifier [0x00007FF6ED47B9A2+3610402]
	GetHandleVerifier [0x00007FF6ED4D1870+3962352]
	GetHandleVerifier [0x00007FF6ED4C9D5F+3930847]
	GetHandleVerifier [0x00007FF6ED1B3656+693206]
	(No symbol) [0x00007FF6ED091638]
	(No symbol) [0x00007FF6ED08D944]
	(No symbol) [0x00007FF6ED08DA72]
	(No symbol) [0x00007FF6ED07E123]
	BaseThreadInitThunk [0x00007FFE07797344+20]
	RtlUserThreadStart [0x00007FFE07D426B1+33]


In [53]:
len(answer_list)

47

- 1 ~ 10 페이지에 대한 답변 가져오기

In [66]:
answer_list = []
driver = webdriver.Chrome()
for page in range(1, 11):
    url = f'https://kin.naver.com/search/list.naver?query={quote("추석 선물")}&page={page}'
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.basic1 > li')

    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
        click_count = (int(answer_num) - 1) // 5 
        print(click_count, end=' ')
        for _ in range(click_count):
            driver.find_element(By.ID, 'nextPageButton').click()
            time.sleep(2)

        sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
        divs = sub_soup.select('.answer-content__list._answerList > div')
        for div in divs:
            ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
            answer = ''
            for p in ps:
                text = p.get_text().strip()
                if text == '\u200b' or text == '':
                    continue
                answer += text + '\n'
            answer_list.append(answer)

        driver.back()
        time.sleep(2)
    print()

driver.close()

0 0 0 0 0 0 0 1 1 3 
1 5 0 1 1 1 3 3 0 0 
1 0 2 4 2 1 0 5 4 0 
2 0 2 6 2 2 2 1 3 4 
3 2 1 0 1 1 0 0 2 1 
1 2 3 0 1 3 2 3 2 1 
1 4 2 1 1 3 3 1 1 2 
1 1 0 0 0 1 1 0 0 5 
3 0 2 0 1 2 0 2 8 0 
0 3 0 0 1 2 3 0 1 0 


In [67]:
len(answer_list)

994

- 파일에 쓰기

In [68]:
with open('data/추석선물.txt', 'w', encoding='utf-8') as file:
    for answer in answer_list:
        file.write(answer + '\n')